In [1]:
import numpy as np
import copy

In [2]:
def sigmoid(x):
    return 1 /(1 + np.exp(-x))

def derivativeSigmoid(x):
    return x * (1 - x)

In [3]:
binary_dim = 8
largest_num = pow(2, binary_dim)
int2bin = {}

In [4]:
largest_num

256

In [5]:
binary = np.unpackbits(np.array([range(largest_num)], dtype = np.uint8).T, axis = 1)

In [6]:
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [7]:
binary.shape

(256, 8)

In [8]:
binary[5]

array([0, 0, 0, 0, 0, 1, 0, 1], dtype=uint8)

In [9]:
for i in range(largest_num):
    int2bin[i] = binary[i]

In [10]:
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1

In [11]:
wh = np.random.random((input_dim, hidden_dim))
wout = np.random.random((hidden_dim, output_dim))
wtime = np.random.random((hidden_dim, hidden_dim))

In [12]:
wh_update = np.zeros_like(wh)
wout_update = np.zeros_like(wout)
wtime_update = np.zeros_like(wtime)

In [ ]:
epochs =100000

for epoch in range(epochs):
    a_int = np.random.randint(largest_num / 2)
    b_int = np.random.randint(largest_num / 2)
    a = int2bin[a_int]
    b = int2bin[b_int]
    
    c_int = a_int + b_int
    c = int2bin[c_int]
    
    d = np.zeros_like(c)
    overallError = 0
    
    layer_1_values = []
    layer_2_deltas = []
    layer_1_values.append(np.zeros(hidden_dim))
    
    for position in range(binary_dim):
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position -1]]])
        
        #hidden layer
        layer_1 = sigmoid(np.dot(X, wh) + np.dot(layer_1_values[-1], wtime))
        
        #output layer
        layer_2 = sigmoid(np.dot(layer_1, wout))
        
        output_error = y - layer_2
        output_slope = derivativeSigmoid(layer_2[0][0])
        layer_2_deltas.append(output_error * output_slope)
        
        
        overallError += np.abs(output_error[0])
        
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        layer_1_values.append(copy.deepcopy(layer_1))
        
    future_layer_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position -1]
        prev_layer_1 = layer_1_values[-position - 2]
        
        delta_output = layer_2_deltas[-position - 1]
        
        hidden_error = future_layer_delta.dot(wtime.T) + delta_output.dot(wout.T)
        slope_hidden = derivativeSigmoid(layer_1)
        hidden_delta = hidden_error * slope_hidden
        
        wout_update += np.atleast_2d(layer_1).T.dot(delta_output)
        wtime_update += np.atleast_2d(prev_layer_1).T.dot(hidden_delta)
        wh_update += X.T.dot(hidden_delta)
        
        future_layer_delta = hidden_delta
        
    wh += wh_update * alpha
    wtime += wtime_update * alpha
    wout += wout_update * alpha
    
    wh_update *= 0
    wtime_update *= 0
    wout_update *= 0
    
    if(epoch % 100 == 0):
        print("Error : {}".format(overallError))
        print("Pred : {}".format(d))
        print("Actual : {}".format(c))
        out = 0
        for index,x in enumerate(reversed(d)):
            out += X * pow(2, index)
            
        print("Actual -> {} + {} = {}".format(a_int, b_int, c_int))
        print("Predicted -> {} + {} = {} ".format(a_int,b_int,c_int))
        print("*" * 20)
        

Error : [3.00996855]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 1 1 0 1 1]
Actual -> 107 + 48 = 155
Predicted -> 107 + 48 = 155 
********************
Error : [5.95921883]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 0 0 0 1 0]
Actual -> 54 + 76 = 130
Predicted -> 54 + 76 = 130 
********************
Error : [4.15950468]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 1 0 1 0 1 1 0]
Actual -> 117 + 97 = 214
Predicted -> 117 + 97 = 214 
********************
Error : [3.98589722]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 0 0 1 1 0 1 0]
Actual -> 63 + 91 = 154
Predicted -> 63 + 91 = 154 
********************
Error : [3.78278697]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 0 0 0 0 1 1 0]
Actual -> 66 + 68 = 134
Predicted -> 66 + 68 = 134 
********************
Error : [4.67174663]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 0 1 1 1 1 1 0]
Actual -> 112 + 78 = 190
Predicted -> 112 + 78 = 190 
********************
Error : [4.39225114]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 0 1 1 1 1 0 0]
Actual -> 105 + 83 = 188
Predicted -> 105 + 8

Error : [4.07445368]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 0 1 0 1 0]
Actual -> 19 + 87 = 106
Predicted -> 19 + 87 = 106 
********************
Error : [3.92780025]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 1 0 1 0 0 1]
Actual -> 81 + 88 = 169
Predicted -> 81 + 88 = 169 
********************
Error : [3.69446362]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 1 1 1 0 0 1]
Actual -> 92 + 93 = 185
Predicted -> 92 + 93 = 185 
********************
Error : [3.12166695]
Pred : [0 0 0 0 0 0 0 0]
Actual : [0 1 0 0 0 1 0 0]
Actual -> 37 + 31 = 68
Predicted -> 37 + 31 = 68 
********************
Error : [4.13650869]
Pred : [0 0 0 0 0 0 0 0]
Actual : [1 1 0 1 0 1 1 0]
Actual -> 106 + 108 = 214
Predicted -> 106 + 108 = 214 
********************
Error : [4.01166318]
Pred : [0 0 0 0 0 0 0 1]
Actual : [0 1 0 0 1 1 0 0]
Actual -> 39 + 37 = 76
Predicted -> 39 + 37 = 76 
********************
Error : [3.8823037]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 1 0 1 0 1 1]
Actual -> 66 + 105 = 171
Predicted -> 66 + 105 = 171

Error : [3.68998319]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 1 0 0 1 0 1 1]
Actual -> 83 + 120 = 203
Predicted -> 83 + 120 = 203 
********************
Error : [3.8298323]
Pred : [0 0 0 0 0 0 0 1]
Actual : [1 0 0 0 0 1 1 1]
Actual -> 121 + 14 = 135
Predicted -> 121 + 14 = 135 
********************
Error : [3.89123933]
Pred : [1 1 1 1 1 1 0 1]
Actual : [1 1 0 1 0 1 0 0]
Actual -> 104 + 108 = 212
Predicted -> 104 + 108 = 212 
********************
Error : [3.4327273]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 0 0 1 1 1]
Actual -> 103 + 0 = 103
Predicted -> 103 + 0 = 103 
********************
Error : [3.677172]
Pred : [0 0 1 1 1 1 1 1]
Actual : [0 1 0 0 1 1 1 1]
Actual -> 28 + 51 = 79
Predicted -> 28 + 51 = 79 
********************
Error : [4.23534052]
Pred : [1 1 1 1 1 1 1 0]
Actual : [1 1 0 0 0 0 0 0]
Actual -> 125 + 67 = 192
Predicted -> 125 + 67 = 192 
********************
Error : [3.68422992]
Pred : [0 0 0 0 0 0 0 1]
Actual : [0 1 0 1 0 0 0 1]
Actual -> 40 + 41 = 81
Predicted -> 40 + 41 = 8

Error : [2.74677975]
Pred : [1 1 0 1 1 1 1 1]
Actual : [1 0 0 1 0 0 1 1]
Actual -> 76 + 71 = 147
Predicted -> 76 + 71 = 147 
********************
Error : [2.73562335]
Pred : [0 1 0 0 0 0 0 1]
Actual : [0 1 1 0 0 0 0 1]
Actual -> 37 + 60 = 97
Predicted -> 37 + 60 = 97 
********************
Error : [1.59180178]
Pred : [0 1 1 1 0 0 0 1]
Actual : [0 1 1 1 0 0 0 1]
Actual -> 41 + 72 = 113
Predicted -> 41 + 72 = 113 
********************
Error : [2.77306141]
Pred : [1 0 0 0 0 0 0 0]
Actual : [1 0 1 0 0 0 0 0]
Actual -> 109 + 51 = 160
Predicted -> 109 + 51 = 160 
********************
Error : [3.28732861]
Pred : [1 0 0 0 0 0 0 1]
Actual : [1 0 1 0 0 1 0 1]
Actual -> 47 + 118 = 165
Predicted -> 47 + 118 = 165 
********************
Error : [3.66446383]
Pred : [1 0 1 0 0 0 0 1]
Actual : [1 0 0 1 1 1 1 1]
Actual -> 77 + 82 = 159
Predicted -> 77 + 82 = 159 
********************
Error : [2.09938311]
Pred : [0 1 1 1 0 1 0 1]
Actual : [0 1 1 1 0 0 1 1]
Actual -> 19 + 96 = 115
Predicted -> 19 + 96 = 11

Error : [2.16510254]
Pred : [0 0 1 1 1 0 1 0]
Actual : [0 0 1 1 1 0 1 0]
Actual -> 58 + 0 = 58
Predicted -> 58 + 0 = 58 
********************
Error : [3.82801922]
Pred : [0 1 1 0 1 1 1 0]
Actual : [0 1 1 1 0 0 0 0]
Actual -> 47 + 65 = 112
Predicted -> 47 + 65 = 112 
********************
Error : [3.66705661]
Pred : [0 1 0 0 1 1 1 0]
Actual : [1 0 1 0 1 1 0 0]
Actual -> 57 + 115 = 172
Predicted -> 57 + 115 = 172 
********************
Error : [2.24917891]
Pred : [1 1 0 1 0 1 0 1]
Actual : [1 0 0 1 0 1 0 1]
Actual -> 114 + 35 = 149
Predicted -> 114 + 35 = 149 
********************
Error : [2.93540537]
Pred : [0 1 0 0 0 1 1 0]
Actual : [0 1 0 1 0 1 1 0]
Actual -> 58 + 28 = 86
Predicted -> 58 + 28 = 86 
********************
Error : [2.0000723]
Pred : [0 1 1 0 0 0 1 1]
Actual : [0 1 1 0 0 0 1 1]
Actual -> 1 + 98 = 99
Predicted -> 1 + 98 = 99 
********************
Error : [4.58495135]
Pred : [0 1 1 1 1 1 0 0]
Actual : [0 1 0 0 0 0 1 0]
Actual -> 11 + 55 = 66
Predicted -> 11 + 55 = 66 
********

Error : [2.91264023]
Pred : [1 1 0 0 0 0 0 1]
Actual : [1 0 1 0 0 0 0 1]
Actual -> 110 + 51 = 161
Predicted -> 110 + 51 = 161 
********************
Error : [2.28987402]
Pred : [0 1 1 0 1 0 1 0]
Actual : [0 1 1 0 0 1 1 0]
Actual -> 86 + 16 = 102
Predicted -> 86 + 16 = 102 
********************
Error : [2.96792544]
Pred : [0 1 1 1 1 1 1 0]
Actual : [0 1 1 0 0 0 0 0]
Actual -> 5 + 91 = 96
Predicted -> 5 + 91 = 96 
********************
Error : [2.65321453]
Pred : [1 1 0 1 1 1 0 0]
Actual : [1 0 0 1 1 1 0 0]
Actual -> 47 + 109 = 156
Predicted -> 47 + 109 = 156 
********************
Error : [2.27914899]
Pred : [1 1 0 1 1 1 1 0]
Actual : [1 1 0 1 1 0 0 0]
Actual -> 113 + 103 = 216
Predicted -> 113 + 103 = 216 
********************
Error : [2.17047862]
Pred : [0 0 1 1 0 1 1 1]
Actual : [0 1 1 1 0 1 1 1]
Actual -> 22 + 97 = 119
Predicted -> 22 + 97 = 119 
********************
Error : [1.08288201]
Pred : [1 0 0 1 1 0 0 1]
Actual : [1 0 0 1 1 0 0 1]
Actual -> 105 + 48 = 153
Predicted -> 105 + 48 

Error : [2.01671291]
Pred : [1 0 0 1 0 0 1 1]
Actual : [1 0 1 0 0 0 1 1]
Actual -> 37 + 126 = 163
Predicted -> 37 + 126 = 163 
********************
Error : [1.02263753]
Pred : [1 0 1 0 1 0 0 0]
Actual : [1 0 1 0 1 0 0 0]
Actual -> 113 + 55 = 168
Predicted -> 113 + 55 = 168 
********************
Error : [1.03904677]
Pred : [1 1 0 1 0 0 0 0]
Actual : [1 1 0 1 0 0 0 0]
Actual -> 91 + 117 = 208
Predicted -> 91 + 117 = 208 
********************
Error : [1.08428726]
Pred : [1 1 0 0 0 1 1 0]
Actual : [1 1 0 0 0 1 1 0]
Actual -> 95 + 103 = 198
Predicted -> 95 + 103 = 198 
********************
Error : [0.4878238]
Pred : [1 0 0 0 0 1 1 1]
Actual : [1 0 0 0 0 1 1 1]
Actual -> 32 + 103 = 135
Predicted -> 32 + 103 = 135 
********************
Error : [3.96154138]
Pred : [1 0 0 1 1 0 1 1]
Actual : [1 1 1 0 0 0 1 1]
Actual -> 125 + 102 = 227
Predicted -> 125 + 102 = 227 
********************
Error : [1.56826115]
Pred : [1 1 0 0 0 0 1 0]
Actual : [1 1 0 0 0 0 0 0]
Actual -> 120 + 72 = 192
Predicted -> 

Error : [0.3944678]
Pred : [0 1 1 1 0 1 1 0]
Actual : [0 1 1 1 0 1 1 0]
Actual -> 2 + 116 = 118
Predicted -> 2 + 116 = 118 
********************
Error : [0.75292325]
Pred : [1 0 0 1 1 0 0 1]
Actual : [1 0 0 1 1 0 0 1]
Actual -> 31 + 122 = 153
Predicted -> 31 + 122 = 153 
********************
Error : [0.33079071]
Pred : [0 1 1 0 0 0 0 0]
Actual : [0 1 1 0 0 0 0 0]
Actual -> 87 + 9 = 96
Predicted -> 87 + 9 = 96 
********************
Error : [0.54003428]
Pred : [1 1 0 1 0 1 1 1]
Actual : [1 1 0 1 0 1 1 1]
Actual -> 127 + 88 = 215
Predicted -> 127 + 88 = 215 
********************
Error : [0.1962749]
Pred : [0 1 0 0 1 1 0 1]
Actual : [0 1 0 0 1 1 0 1]
Actual -> 37 + 40 = 77
Predicted -> 37 + 40 = 77 
********************
Error : [0.51629044]
Pred : [1 1 0 0 1 0 1 1]
Actual : [1 1 0 0 1 0 1 1]
Actual -> 126 + 77 = 203
Predicted -> 126 + 77 = 203 
********************
Error : [0.26506059]
Pred : [0 1 0 0 1 0 0 0]
Actual : [0 1 0 0 1 0 0 0]
Actual -> 2 + 70 = 72
Predicted -> 2 + 70 = 72 
*****

Error : [0.14546432]
Pred : [1 0 1 0 0 1 0 0]
Actual : [1 0 1 0 0 1 0 0]
Actual -> 98 + 66 = 164
Predicted -> 98 + 66 = 164 
********************
Error : [0.14883073]
Pred : [0 1 1 0 0 1 1 0]
Actual : [0 1 1 0 0 1 1 0]
Actual -> 33 + 69 = 102
Predicted -> 33 + 69 = 102 
********************
Error : [0.22522651]
Pred : [0 1 1 0 1 1 0 0]
Actual : [0 1 1 0 1 1 0 0]
Actual -> 105 + 3 = 108
Predicted -> 105 + 3 = 108 
********************
Error : [0.20008015]
Pred : [0 1 1 0 0 1 0 1]
Actual : [0 1 1 0 0 1 0 1]
Actual -> 74 + 27 = 101
Predicted -> 74 + 27 = 101 
********************
Error : [0.13717247]
Pred : [0 1 0 1 0 0 1 1]
Actual : [0 1 0 1 0 0 1 1]
Actual -> 82 + 1 = 83
Predicted -> 82 + 1 = 83 
********************
Error : [0.24612759]
Pred : [1 1 0 1 1 0 1 1]
Actual : [1 1 0 1 1 0 1 1]
Actual -> 107 + 112 = 219
Predicted -> 107 + 112 = 219 
********************
Error : [0.32241698]
Pred : [0 1 1 1 0 0 0 0]
Actual : [0 1 1 1 0 0 0 0]
Actual -> 62 + 50 = 112
Predicted -> 62 + 50 = 112 

Error : [0.1948283]
Pred : [1 0 1 0 1 0 1 0]
Actual : [1 0 1 0 1 0 1 0]
Actual -> 88 + 82 = 170
Predicted -> 88 + 82 = 170 
********************
Error : [0.29766547]
Pred : [1 0 0 1 0 0 1 1]
Actual : [1 0 0 1 0 0 1 1]
Actual -> 54 + 93 = 147
Predicted -> 54 + 93 = 147 
********************
Error : [0.11757951]
Pred : [0 1 0 1 1 1 0 0]
Actual : [0 1 0 1 1 1 0 0]
Actual -> 82 + 10 = 92
Predicted -> 82 + 10 = 92 
********************
Error : [0.21356251]
Pred : [0 1 1 1 1 1 0 1]
Actual : [0 1 1 1 1 1 0 1]
Actual -> 110 + 15 = 125
Predicted -> 110 + 15 = 125 
********************
Error : [0.20761835]
Pred : [1 0 1 0 0 1 1 0]
Actual : [1 0 1 0 0 1 1 0]
Actual -> 113 + 53 = 166
Predicted -> 113 + 53 = 166 
********************
Error : [0.24323217]
Pred : [0 0 1 1 1 0 0 0]
Actual : [0 0 1 1 1 0 0 0]
Actual -> 26 + 30 = 56
Predicted -> 26 + 30 = 56 
********************
Error : [0.36289148]
Pred : [1 1 1 0 0 0 1 0]
Actual : [1 1 1 0 0 0 1 0]
Actual -> 107 + 119 = 226
Predicted -> 107 + 119 = 2